In [423]:
import datetime as dt
from datetime import date
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import time
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

# https://www.youtube.com/watch?v=QIUxPv5PJOY

In [109]:
def format_data(df,start, end):
    training_data = df[start:end].drop('Date', axis = 1)
    return(training_data)
        
def create_xy_scaled(df, samples):
    
    dataset = df.values
    scaler = MinMaxScaler(feature_range = (0,1))
    scaled_data = scaler.fit_transform(dataset)
    data = scaled_data
    x = []
    y = []

    for i in range(samples, len(train_data)):
        x.append(data[i-samples:i])
        y.append(data[i])
        
    return(np.array(x), np.array(y), scaler)

#returns list of RSI values over n periods
def RSI(df, n):
    L = len(df)
    up = [0]*L
    down = [0]*L
    
    for i in range(1,L):
        if df['Close'][i-1] <= df['Close'][i]:
            up[i] = df['Close'][i] - df['Close'][i-1]
        if df['Close'][i-1] > df['Close'][i]:
            down[i] = abs(df['Close'][i] - df['Close'][i-1])

    avg_up= [0]*L
    avg_down = [0]*L
    
    avg_up[n] = np.mean(up[1:n+1])
    avg_down[n] = np.mean(down[1:n+1])
    
    for i in range(n+1,L):
        avg_up[i] = (avg_up[i-1]*(n-1)+up[i])/n
        avg_down[i] = (avg_down[i-1]*(n-1)+down[i])/n
    
    #return(avg_up,avg_down)
    RSI = [0]*L
    for i in range(n,L):
        RSI[i]= 100 - 100/(1+ (avg_up[i-1]/avg_down[i]))
    
    return(RSI)


##SMA, returns list of simple moving average over n periods
def SMA(stock_df,n):
    A = [0]*len(stock_df)
    for i in range(len(stock_df)-n+1):
        A[i+n-1] = np.mean(list(stock_df['Close'][i:i+n]))
    return(A)

#EMA, returns list of exponential moving average over n periods
def EMA(stock_df,n):
    mult = 2/(n+1)
    A = [0]*len(stock_df)
    A[n-1] = SMA(stock_df,n)[n-1]
    for i in range(n-1,len(stock_df)-1):
        A[i+1] = A[i]+(stock_df['Close'][i]-A[i])*mult
    return(A)

In [420]:
QQQ = pd.read_csv('dataset_QQQ_20140207_to_20181231.csv')
QQQ.drop('Date', axis =1, inplace = True)
x = create_xy_scaled(QQQ,60)[0]
y = create_xy_scaled(QQQ,60)[1]
scale = create_xy_scaled(QQQ,60)[2]
x_train = x[:1000].copy()
y_train = y[:1000].copy()

x_test = x[1001:].copy()
y_test = y[1001:].copy()

In [421]:
model = Sequential()
model.add(LSTM(50, return_sequences = True, input_shape = (x_train.shape[1], x_train.shape[2])))
model.add(LSTM(50, return_sequences = False))
model.add(Dense(25))
model.add(Dense(y_train.shape[1]))

model.compile(optimizer = 'adam', loss = 'mean_squared_error')

model.fit(x_train, y_train, batch_size = 1, epochs = 1)

Train on 1000 samples
1000/1000 [==============================] - 40s 40ms/sample - loss: 0.0083


In [425]:
model.save('savetest.model')

In [435]:
def pred(x, model):
    x = x.reshape(1, x.shape[0], x.shape[1])
    return(model.predict(x))

In [374]:
model = tf.keras.models.load_model('savetest.model')

def unscale(data, scaler):
    return(scaler.inverse_transform(data))

def rescale(data, scaler):
    return(scaler.transform(data))

def pred(x, model):
    return(model.predict(x))

In [ ]:
# predict loop test
model = tf.keras.models.load_model('2020-06-23-LSTM-60_to_1.model')
df = create_xy_scaled(QQQ,50)[0][-50:]
scale = create_xy_scaled(QQQ,50)[2]

forecast = 1
for i in range(forecast):
    
    y_pred = pred(df, model)
    
    new_row ={'Close':y_pred}
    QQQ_test.drop([' 9 RSI', '12 EMA', '26 EMA'], axis = 1, inplace = True)
    QQQ_test = QQQ_test.append(new_row, ignore_index = True,)
    QQQ_test[' 9 RSI'] = RSI(QQQ_test, 9)
    QQQ_test['12 EMA'] = EMA(QQQ_test, 12)
    QQQ_test['26 EMA'] = EMA(QQQ_test, 26)

In [388]:
df = QQQ[-50:]